In [ ]:
# Download the data 
!mkdir graph_data
!wget https://www.physi.uni-heidelberg.de/~dittmeier/pytorch/graph_data/graphs_pT2GeV.zip
#wget https://www.physi.uni-heidelberg.de/~dittmeier/pytorch/graph_data/graphs_no_pTCut.zip # uncomment this line to download the data without pT cut

In [ ]:
!unzip graphs_pT2GeV.zip -d graph_data
#!unzip graphs_no_pTCut.zip -d graph_data   # uncomment this line to unzip the data without pT cut

In [ ]:
data_dir = 'graph_data/pTge2GeV'
#data_dir = 'graph_data/nopTCut/' # uncomment this line to use the data without pT cut

In [ ]:
#install required packages
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
#ensure that the PyTorch and the PyG are the same version
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from torch_geometric.data import Data

# Define the path to the PyG file
file_path = f'{data_dir}/trainset/event000021000.pyg'

# Load the PyG file
data = torch.load(file_path)

# Print the properties of the PyG file
print(data)
print(data.num_nodes)
print(data.num_edges)
print(data.edge_index)
print(data.y)
print(data.truth_map)
print(data.track_edges)
print(data.x)
print(data.r)
print(data.phi)
print(data.z)

In [ ]:
from utils.version_utils import get_pyg_data_keys
from utils import (
    load_datafiles_in_dir,
    handle_weighting,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from torch_geometric.data import Dataset

class GraphDataset(Dataset):
    """
    The custom default GNN dataset to load graphs off the disk
    """

    def __init__(
        self,
        input_dir,
        data_name=None,
        num_events=None,
        stage="fit",
        hparams=None,
        transform=None,
        pre_transform=None,
        pre_filter=None,
        preprocess=True,
    ):
        if hparams is None:
            hparams = {}
        super().__init__(input_dir, transform, pre_transform, pre_filter)

        self.input_dir = input_dir
        self.data_name = data_name
        self.hparams = hparams
        self.num_events = num_events
        self.stage = stage
        self.preprocess = preprocess
        self.transform = transform

        self.input_paths = load_datafiles_in_dir(
            self.input_dir, self.data_name, self.num_events
        )
        self.input_paths.sort()  # We sort here for reproducibility

    def len(self):
        return len(self.input_paths)

    def get(self, idx):
        event_path = self.input_paths[idx]
        event = torch.load(event_path, map_location=torch.device("cpu"))
        # convert DataBatch to Data instance because some transformations don't work on DataBatch
        event = Data(**event.to_dict())
        if not self.preprocess:
            return event
        event = self.preprocess_event(event)
        # do pyg transformation if a torch_geometric.transforms instance is given
        if self.transform is not None:
            event = self.transform(event)

        # return (event, event_path) if self.stage == "predict" else event
        return event
    
    def preprocess_event(self, event):
        """
        Process event before it is used in training and validation loops
        """
        event = self.construct_weighting(event)
        event = self.scale_features(event)
        return event

    def construct_weighting(self, event):
        """
        Construct the weighting for the event
        """

        assert event.y.shape[0] == event.edge_index.shape[1], (
            f"Input graph has {event.edge_index.shape[1]} edges, but"
            f" {event.y.shape[0]} truth labels"
        )

        if self.hparams is not None and "weighting" in self.hparams.keys():
            assert isinstance(self.hparams["weighting"], list) & isinstance(
                self.hparams["weighting"][0], dict
            ), "Weighting must be a list of dictionaries"
            event.weights = handle_weighting(event, self.hparams["weighting"])
        else:
            event.weights = torch.ones_like(event.y, dtype=torch.float32)

        return event


    def scale_features(self, event):
        """
        Handle feature scaling for the event
        """

        if (
            self.hparams is not None
            and "node_scales" in self.hparams.keys()
            and "node_features" in self.hparams.keys()
        ):
            assert isinstance(
                self.hparams["node_scales"], list
            ), "Feature scaling must be a list of ints or floats"
            for i, feature in enumerate(self.hparams["node_features"]):
                assert feature in get_pyg_data_keys(
                    event
                ), f"Feature {feature} not found in event"
                event[feature] = event[feature] / self.hparams["node_scales"][i]

        return event

In [ ]:
# weighting for the different classes
# false: edges that we want to get rid of
weighting_false = {
    "weight": # choose a weight (floating point value >= 0)
    "conditions": {
        "y": False
    }
}

# bckg: edges that are not of interest
weighting_bckg = {
    "weight": # choose a weight (floating point value >= 0)
    "conditions": {
        "y": True
    }
}

# target: edges that are of interest, we want to keep them
weighting_target = {
    "weight": # choose a weight (floating point value >= 0)
    "conditions": {
        "y": True,
        "pt": [1., float('inf')],
        "nhits": [3, float('inf')]
    }
}

# hyperparameters for the dataset
parameters = {
    "node_features": ["r",  "phi", "z"],
    "node_scales": [1000, 3.14,  1000],
    "weighting": [weighting_false,
                  weighting_bckg,
                  weighting_target], # list of dictionaries
}


trainset = GraphDataset(f"{data_dir}/trainset", hparams=parameters)
valset = GraphDataset(f"{data_dir}/valset", hparams=parameters)
testset = GraphDataset(f"{data_dir}/testset", hparams=parameters)
print("Number of samples in trainset:", len(trainset))
print("Number of samples in valset:", len(valset))
print("Number of samples in testset:", len(testset))


In [ ]:
from torch_geometric.loader import DataLoader

# Set batch size and number of workers
batch_size = 1
num_workers = 1

# Create dataloaders
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

for test_event in train_loader:
    print(test_event.r)
    print(test_event.y.shape)
    print(test_event.weights.shape)
    print(test_event.weights)
    num_zero_weights = (test_event.weights == 0).sum().item()   
    print("Number of 0 weights:", num_zero_weights)
    num_point1_weights = (test_event.weights == 0.1).sum().item()
    print("Number of 0.1 weights:", num_point1_weights)
    num_one_weights = (test_event.weights == 1).sum().item()
    print("Number of 1 weights:", num_one_weights)
    
    break

In [ ]:
from torch_geometric.nn import MessagePassing
import torch
import torch.nn as nn

# https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_gnn.html

class InteractionConv(MessagePassing):
    def __init__(
        self,
        hidden_dim,
        aggr="add",
        *,
        aggr_kwargs={},
        flow: str = "source_to_target",
        node_dim: int = -2,
        decomposed_layers: int = 1,
        **kwargs
    ):
        super().__init__(
            aggr,
            aggr_kwargs=aggr_kwargs,
            flow=flow,
            node_dim=node_dim,
            decomposed_layers=decomposed_layers,
            **kwargs
        )
        
        # example network for node updates; can be replaced with any other network. Experiment here!
        self.node_network = nn.Sequential(
            nn.Linear(2*hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # example network for edge updates; can be replaced with any other network. Experiment here!
        self.edge_network = nn.Sequential(
            nn.Linear(3*hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

    def message(self,e):
        # constructs messages for each edge; e as given to propagate
        return e

    def aggregate(
        self,
        inputs: torch.Tensor,
        index: torch.Tensor,
        edge_index,
        x,
        ptr=None,
        dim_size=None,
    ) -> torch.Tensor:
        # takes output from message as first argument (inputs = e), and can take any other data passed to propagate
        # so the edge data is creating a message for source and destination nodes
        src_message = self.aggr_module(inputs, edge_index[1], dim_size=x.size(0))
        dst_message = self.aggr_module(inputs, edge_index[0], dim_size=x.size(0))
        out = src_message + dst_message
        return out

    def update(self, inputs: torch.Tensor, x) -> torch.Tensor:
        # takes the aggregated messages and the node data and updates the node data
        x_in = torch.cat([x, inputs], dim=1)
        out = self.node_network(x_in)
        return out

    def edge_update(self, edge_index, x, e) -> torch.Tensor:
        x_in = torch.cat([x[edge_index[0]], x[edge_index[1]], e], dim=1)
        out = self.edge_network(x_in)
        return out

    def forward(self, edge_index, x, e):
        # propagate: initial call to start propagating messages, takes edge indices and any other data
        # propagate calls message, aggreate and update functions
        x = self.propagate(edge_index=edge_index, x=x, e=e)
        # then we update our edge features, this calls edge_update
        e = self.edge_updater(edge_index=edge_index, x=x, e=e)
        return x, e



In [ ]:
class InteractionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_iterations=2):
        super(InteractionNetwork, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_iterations = n_iterations
        
        # example network for node encoding; can be replaced with any other network. Experiment here!
        self.node_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # example network for edge encoding; can be replaced with any other network. Experiment here!
        self.edge_encoder = nn.Sequential(
            nn.Linear(2*hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.conv = InteractionConv(hidden_dim)
        
        # example network for edge classification; can be replaced with any other network. Experiment here!
        self.edge_classifier = nn.Sequential(
            nn.Linear(3* hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    # we take a full data batch, and make use of features we want 
    def forward(self, batch):
        # Extract node features
        x = torch.stack([batch.r, batch.phi, batch.z], dim=-1).to(torch.float32)
        edge_index = batch.edge_index
        #print(f"x= {x}")
        #print(f"edge_index= {edge_index.shape}")
        
        #if "undirected" in self.hparams and self.hparams["undirected"]:
        #edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
        #print(f"edge_index= {edge_index.shape}")

        start, end = edge_index
        x.requires_grad = True
        #print(start, end)

        x = self.node_encoder(x)
        e = self.edge_encoder(torch.cat([x[start], x[end]], dim=-1))
        #print(x)
        #print(e)
        # Message passing
        for i in range(self.n_iterations):
            x, e = self.conv(edge_index, x, e)
            
        #return
        # Decode edge features
        decoded = self.edge_classifier(torch.cat([x[start], x[end], e], dim=-1))
        
        #print(decoded)
        return decoded


In [ ]:
input_dim = 3 # currently assumes r, phi and z in the forward pass 
# --> (3 input dimensions; you can change this, but you need to adjust the forward pass as well!)
hidden_dim = # choose  hidden dimensions: experiment here
n_iterations = # choose Number of message passing steps: experiment here

model = InteractionNetwork(input_dim=input_dim, hidden_dim=hidden_dim, n_iterations=n_iterations).to(device)
print(test_event)
test_run = model(test_event.to(device))
print(test_run)


In [ ]:
learning_rate = 1e-3 # experiment here
epochs = 20 # experiment here
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # example optimizer, experiment here
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) # example scheduler, experiment here

In [ ]:
import torch.nn.functional as F

def loss_function(output, batch):
        """
        Applies the loss function to the output of the model and the truth labels.
        To balance the positive and negative contribution, simply take the means of each separately.
        Any further fine tuning to the balance of true target, true background and fake can be handled
        with the `weighting` config option.
        """

        assert hasattr(batch, "y"), (
            "The batch does not have a truth label. Please ensure the batch has a `y`"
            " attribute."
        )
        assert hasattr(batch, "weights"), (
            "The batch does not have a weighting label. Please ensure the batch"
            " weighting is handled in preprocessing."
        )

        negative_mask = ((batch.y == 0) & (batch.weights != 0)) | (batch.weights < 0)
        #print(output[negative_mask].squeeze().shape)
        #print(torch.zeros_like(output[negative_mask]).squeeze().shape)
        #print(batch.weights[negative_mask].abs().shape)

        negative_loss = F.binary_cross_entropy_with_logits(
            output[negative_mask].squeeze(),
            torch.zeros_like(output[negative_mask]).squeeze(),
            weight=batch.weights[negative_mask].abs(),
            reduction="sum",
        )

        positive_mask = (batch.y == 1) & (batch.weights > 0)
        positive_loss = F.binary_cross_entropy_with_logits(
            output[positive_mask].squeeze(),
            torch.ones_like(output[positive_mask]).squeeze(),
            weight=batch.weights[positive_mask].abs(),
            reduction="sum",
        )

        n = positive_mask.sum() + negative_mask.sum()
        return (
            (positive_loss + negative_loss) / n,
            positive_loss.detach() / n,
            negative_loss.detach() / n,
        )

In [ ]:
# loops over our optimization code
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    print(f"current learning rate: {optimizer.param_groups[0]['lr']}")
    for batchid, batch in enumerate(dataloader):

        output = model(batch.to(device))
        loss, pos_loss, neg_loss = loss_fn(output, batch)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batchid % 10 == 0:
            loss, current = loss.item(), batchid * batch_size + len(batch.event_id)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        # Clear GPU memory
        del batch, output, loss
        torch.cuda.empty_cache()
    scheduler.step()



In [ ]:
# evaluate the model's performance against the test dataset
def test_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    test_loss = 0
    with torch.no_grad():   
        for batchid, batch in enumerate(dataloader):

            output = model(batch.to(device))
            loss, pos_loss, neg_loss = loss_fn(output, batch)
            test_loss += loss

            scores = torch.sigmoid(output.squeeze())
            #print(scores)
            efficiencies = []
            purities = []
            for i in range(0, 100, 5):
                score_cut = i / 100
                score_cut_mask = (scores > score_cut)
                positive_mask = (batch.y == 1) & (batch.weights > 0)    # only checking for target edges
                scored_positive = (score_cut_mask & positive_mask).sum()
                #print(f"scored_negative: {scored_negative}")
                #print(f"scored_positive: {scored_positive}")
                efficiency = scored_positive / positive_mask.sum()
                purity = scored_positive / score_cut_mask.sum()
                efficiencies.append(efficiency.item())
                purities.append(purity.item())
                #print(f"efficiency: {efficiency}")
                #print(f"purity: {purity}")
            

            # Clear GPU memory
            del batch, output, loss, scores, score_cut_mask, positive_mask
            torch.cuda.empty_cache()
    test_loss /= size
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss.item(), efficiencies, purities


In [ ]:
loss = []
eff  = []
pur  = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_function, optimizer, device)
    l, e, p = test_loop(val_loader, model, loss_function, optimizer, device)
    loss.append(l)
    eff.append(e)
    pur.append(p)
print("Done!")

In [ ]:
#for name, param in model.named_parameters():
#    print(f"Parameter name: {name}, Size: {param.size()}, Values: {param}")

# Assuming you have the `test_loss` variable containing the loss values for each epoch
import numpy as np

epochs_r = range(1, len(loss) + 1)
plt.figure(figsize=(6, 4))
plt.plot(epochs_r, loss, 'b', label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Validation loss')
plt.legend()
plt.show()

score_cut = np.arange(0, 1 ,0.05)
print(score_cut)
plt.figure(figsize=(6, 4))
plt.xlabel('Score Cut')
plt.ylabel('Efficiency')
for i, y in enumerate(eff):
    if i % 10 == 0 or i == len(pur)-1:
        plt.plot(score_cut, y, label=f'Efficiency after epoch {i+1}')
#plt.ylim(0.9, 1)
plt.legend()
plt.show()
    
score_cut = np.arange(0, 1 ,0.05)
print(score_cut)
plt.figure(figsize=(6, 4))
plt.xlabel('Score Cut')
plt.ylabel('Purity')
for i, y in enumerate(pur):
    if i % 10 == 0 or i == len(pur)-1:
        plt.plot(score_cut, y, label=f'Purity after epoch {pur.index(y)+1}')
#plt.ylim(0.5, 1)
plt.legend()
plt.show()

# print the final efficiency and purity at 0.05 score cut
print(f"Efficiency: {eff[-1][1]}")
print(f"Purity: {pur[-1][1]}")
# and at 0.5 score cut --> this would be the ideal operating point
print(f"Efficiency: {eff[-1][10]}")
print(f"Purity: {pur[-1][10]}")